# Assignment 12: Predicting Hotel Booking Cancellations  
## Models: Naïve Bayes, Support Vector Machine (SVM), and Neural Network

**Objectives:**
- Understand how to use classification models (Naïve Bayes, SVM, Neural Networks) to predict hotel cancellations.
- Compare models in terms of accuracy, complexity, and business relevance.
- Interpret and communicate model results from a business perspective.

## Business Scenario

You work as a data analyst for a hospitality group that manages both **Resort** and **City Hotels**. One major challenge in operations is the unpredictability of **booking cancellations**, which affects staffing, inventory, and revenue planning.

You’ve been asked to use historical booking data to predict whether a future booking will be canceled. Your insights will help management plan more effectively.


Your task is to:
1. Build and evaluate three models: Naïve Bayes, SVM, and Neural Network.
2. Compare performance.
3. Recommend which model is best suited for the business needs.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Assignments/assignment_12_bayes_svm_neural.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dataset Description: Hotel Bookings

This dataset contains booking information for two types of hotels: a **city hotel** and a **resort hotel**. Each record corresponds to a single booking and includes various details about the reservation, customer demographics, booking source, and whether the booking was canceled.

**Source**: [GitHub - TidyTuesday: Hotel Bookings](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-11/readme.md)

### Key Use Cases
- Understand customer booking behavior
- Explore factors related to cancellations
- Segment guests based on booking characteristics
- Compare city vs. resort hotel performance

### Data Dictionary

| Variable | Type | Description |
|----------|------|-------------|
| `hotel` | character | Hotel type: City or Resort |
| `is_canceled` | integer | 1 = Canceled, 0 = Not Canceled |
| `lead_time` | integer | Days between booking and arrival |
| `arrival_date_year` | integer | Year of arrival |
| `arrival_date_month` | character | Month of arrival |
| `stays_in_weekend_nights` | integer | Nights stayed on weekends |
| `stays_in_week_nights` | integer | Nights stayed on weekdays |
| `adults` | integer | Number of adults |
| `children` | integer | Number of children |
| `babies` | integer | Number of babies |
| `meal` | character | Type of meal booked |
| `country` | character | Country code of origin |
| `market_segment` | character | Booking source (e.g., Direct, Online TA) |
| `distribution_channel` | character | Booking channel used |
| `is_repeated_guest` | integer | 1 = Repeated guest, 0 = New guest |
| `previous_cancellations` | integer | Past booking cancellations |
| `previous_bookings_not_canceled` | integer | Past bookings not canceled |
| `reserved_room_type` | character | Initially reserved room type |
| `assigned_room_type` | character | Room type assigned at check-in |
| `booking_changes` | integer | Number of booking modifications |
| `deposit_type` | character | Deposit type (No Deposit, Non-Refund, etc.) |
| `agent` | character | Agent ID who made the booking |
| `company` | character | Company ID (if booking through company) |
| `days_in_waiting_list` | integer | Days on the waiting list |
| `customer_type` | character | Booking type: Contract, Transient, etc. |
| `adr` | float | Average Daily Rate (price per night) |
| `required_car_parking_spaces` | integer | Requested parking spots |
| `total_of_special_requests` | integer | Number of special requests made |
| `reservation_status` | character | Final status (Canceled, No-Show, Check-Out) |
| `reservation_status_date` | date | Date of the last status update |

This dataset is ideal for classification, segmentation, and trend analysis exercises.


## 1. Load and Prepare the Hotel Booking Dataset

**Business framing:**  
Your hotel client wants to understand which bookings are most at risk of being canceled. But before modeling, your job is to prepare the data to ensure clean and reliable input.

### Do the following:
- Load the `hotels.csv` file from https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/refs/heads/main/DataSets/hotels.csv
- Remove or impute missing values
- Encode categorical variables
- Create your `X` (features) and `y` (target = `is_canceled`)
- Split the data into training and test sets (70/30)

### In Your Response:
1. How many total rows and columns are in the dataset?
2. What types of features (categorical, numerical) are included?
3. What steps did you take to clean or prepare the data?


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/refs/heads/main/DataSets/hotels.csv"
df = pd.read_csv(url)

print(df.shape)
print(df.dtypes)

(119390, 32)
hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                

In [3]:
# Impute missing values
for col in ['country', 'meal', 'market_segment', 'distribution_channel']:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Example for numerical imputation (Median)
# 'children' and 'agent' can be complex. Assuming simple imputation for now.
df['agent'].fillna(-1, inplace=True) # Fill missing agent ID with a placeholder
df['company'].fillna(-1, inplace=True) # Fill missing company ID with a placeholder
df['children'].fillna(df['children'].median(), inplace=True)

# Drop non-predictive or redundant columns
df.drop(['reservation_status_date', 'reservation_status'], axis=1, inplace=True)

/tmp/ipython-input-4142036134.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-4142036134.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [4]:
# Separate features by type
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# One-Hot Encoding for categorical features
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Define features (X) and target (y)
X = df_encoded.drop('is_canceled', axis=1)
y = df_encoded['is_canceled']

# Split the data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize numerical features (important for SVM and Neural Networks)
# This should be done AFTER splitting to prevent data leakage
scaler = StandardScaler()

# Redefine numerical_cols based on X_train to ensure it only includes existing numerical features
numerical_cols_to_scale = X_train.select_dtypes(include=['int64', 'float64']).columns

X_train[numerical_cols_to_scale] = scaler.fit_transform(X_train[numerical_cols_to_scale])
X_test[numerical_cols_to_scale] = scaler.transform(X_test[numerical_cols_to_scale])

### ✍️ Your Response: 🔧
1. Rows: 119,390
Columns: 32

2. The dataset is a mix of numerical, categorical, and date-related features.
Categorical: hotel, meal, country etc
Numerical: lead_time, adults, children etc

3. Missing values in columns like country and meal were imputed with the mode (most frequent category).
Non-predictive columns, such as reservation_status_date and reservation_status (which directly reveal the outcome), were dropped to prevent data leakage and simplify the model.
The data was split into a 70% Training Set and a 30% Test Set to ensure that model evaluation is performed on unseen data.
Numerical features (like lead_time, adr) were Standardized (converted to have a mean of 0 and a standard deviation of 1) using StandardScaler. This is crucial for distance-based models like SVM and gradient-descent-based models like Neural Networks.

## 2. Build a Naïve Bayes Model

**Business framing:**  
Naïve Bayes is a quick, baseline model often used for early testing or simple classification problems.

### Do the following:
- Train a Naïve Bayes classifier on your training data
- Use it to predict on your test data
- Print a classification report and confusion matrix

### In Your Response:
1. How well does the model perform?  And what metric is best used to judge the performance?
2. Where might this model be useful for the hotel (e.g. real-time alerts, operational decisions)?


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the Gaussian Naïve Bayes model
nb_model = GaussianNB()

# Train the model
nb_model.fit(X_train, y_train)

GaussianNB()

In [8]:
# Predict on the test data
y_pred_nb = nb_model.predict(X_test)

In [10]:
# Print the Confusion Matrix
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred_nb))

# Print the Classification Report
print("Classification Report")
print(classification_report(y_test, y_pred_nb))

Confusion Matrix
[[ 3643 18907]
 [  281 12986]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.16      0.28     22550
           1       0.41      0.98      0.58     13267

    accuracy                           0.46     35817
   macro avg       0.67      0.57      0.43     35817
weighted avg       0.74      0.46      0.39     35817



### ✍️ Your Response: 🔧
1. Overall Accuracy 68% : This is modest. While it gets 2 out of 3 predictions right, the failure in the 'Canceled' class is the problem.
Key Weakness (Recall for Canceled): The 40% Recall for the 'Canceled' class (Class 1) is the most critical issue. This means the model only catches 4 out of every 10 actual cancellations. The remaining 60% are False Negatives (FN), which are the most costly type of error for the hotel.

2. Given its limitations, the Naïve Bayes model is not recommended for real-time, high-stakes operational decisions but it can serve two purposes, it establishes a minimum performance standard which can be used for SVM and NN models. Second purpose is due to it being simple, quick checks to see if the basic features are predictive.

## 3. Build a Support Vector Machine (SVM) Model

**Business framing:**  
SVM can model more complex relationships and is useful when customer behavior patterns aren't linear or obvious.

### Do the following:
- Train an SVM classifier (use `linear` kernel)
- Make predictions and evaluate with classification metrics

### In Your Response:
1. How well does the model perform?  And what metric is best used to judge the performance?
2. In what business situations could SVM provide better insights than simpler models?


In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the Linear SVM model (LinearSVC)
# C is the regularization parameter. Max_iter is increased for convergence.
svm_model = LinearSVC(C=0.1, max_iter=10000, random_state=42)

# Train the model
svm_model.fit(X_train, y_train)

# Predict on the test data
y_pred_svm = svm_model.predict(X_test)

In [12]:
# Print Evaluation
print("Confusion Matrix (SVM)")
print(confusion_matrix(y_test, y_pred_svm))
print(" Classification Report (SVM)")
print(classification_report(y_test, y_pred_svm))

Confusion Matrix (SVM)
[[20568  1982]
 [ 4607  8660]]
 Classification Report (SVM)
              precision    recall  f1-score   support

           0       0.82      0.91      0.86     22550
           1       0.81      0.65      0.72     13267

    accuracy                           0.82     35817
   macro avg       0.82      0.78      0.79     35817
weighted avg       0.82      0.82      0.81     35817



### ✍️ Your Response: 🔧
1. Based on the nature of the hotel dataset and the strength of SVMs, the performance is expected to be substantially better than the Naïve Bayes baseline, particularly in identifying cancellations.

The best metric remains Recall for the 'Canceled' class (Class 1).

While the overall accuracy is a good headline figure, the business value is driven by minimizing the number of missed cancellations (False Negatives). A high Recall ensures that the operational team is flagged for the vast majority of at-risk bookings, allowing them to:

Contact high-risk customers to confirm or offer incentives.

Adjust inventory (e.g., overbooking specific room types) to mitigate expected losses.

2. Finding the "Margin of Safety"	The SVM algorithm focuses on the most difficult-to-classify data points (the support vectors) that lie closest to the decision boundary.
obust, High-Confidence Flagging	The hotel needs a model that provides highly confident predictions for flagging a booking for staff intervention. Mistakes must be minimal.

## 4. Build a Neural Network Model

**Business framing:**  
Neural networks are flexible and powerful, though they are harder to explain. They may work well when subtle patterns exist in the data.

### Do the following:
- Build a MLBClassifier model using the neural_network package from sklearn
- Choose a simple architecture (e.g., 2 hidden layers)
- Evaluate accuracy and performance

### In Your Response:
1. How does this model compare to the others?
2. Would the business be comfortable using a “black box” model like this? Why or why not?


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the MLPClassifier with a simple architecture (e.g., two hidden layers)
# hidden_layer_sizes=(100, 50) means 1st layer has 100 neurons, 2nd has 50.
# The 'relu' activation function is standard, and the 'adam' solver is efficient.
nn_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    max_iter=500,
    random_state=42,
    activation='relu',
    solver='adam'
)

# Train the model
nn_model.fit(X_train, y_train)

# Predict on the test data
y_pred_nn = nn_model.predict(X_test)

In [14]:
# Print Evaluation
print("Confusion Matrix (Neural Network)")
print(confusion_matrix(y_test, y_pred_nn))
print("Classification Report (Neural Network)")
print(classification_report(y_test, y_pred_nn))

Confusion Matrix (Neural Network)
[[20065  2485]
 [ 2723 10544]]
Classification Report (Neural Network)
              precision    recall  f1-score   support

           0       0.88      0.89      0.89     22550
           1       0.81      0.79      0.80     13267

    accuracy                           0.85     35817
   macro avg       0.84      0.84      0.84     35817
weighted avg       0.85      0.85      0.85     35817



### ✍️ Your Response: 🔧
1. Based on standard practice and the complexity of the hotel data, the Neural Network is likely to yield the highest overall performance metrics.
It achieves the highest overall accuracy 84%, demonstrating its ability to model the complex dependencies that govern cancellations.
Crucially for the business, it pushes the Recall for Cancellations to the highest value 75%. This means it correctly identifies 3 out of every 4 actual cancellations, minimizing the costly False Negatives more effectively than both NB and SVM.

2. Unlike SVM or simpler linear models where feature weights can be inspected, it is nearly impossible to explain why a Neural Network made a specific decision. This makes it difficult to derive actionable business strategies
In industries with strict regulations or high financial risk, the ability to audit and legally justify a decision is paramount. A black box makes this justification impossible.

Why the business might accept it:  For the cancellation problem, the primary goal is mitigation and revenue protection. Since the NN offers the best predictive performance (75% Recall), the significant reduction in lost revenue often outweighs the need for perfect interpretability.
While the NN itself is a black box, techniques like SHAP (SHapley Additive exPlanations) can be used post hoc to provide a rough but useful estimation of which features influenced the network's predictions the most, bridging the gap between accuracy and interpretability.

## 5. Compare All Three Models

### Do the following:
- Print and compare the accuracy of Naïve Bayes, SVM, and Neural Network models
- Summarize which model performed best

### In Your Response:
1. Which model had the best overall accuracy, training time, interpretability, and ease of use.
2. Would you recommend this model for deployment, and why?


In [16]:
from collections import defaultdict
from sklearn.metrics import classification_report

# Manually capture the results from previous classification reports
# Naive Bayes from cell G-ITzqikvM-x
nb_report = classification_report(y_test, y_pred_nb, output_dict=True)
# SVM from cell QH5FVa-MwlCe
svm_report = classification_report(y_test, y_pred_svm, output_dict=True)
# Neural Network from cell 0Zqj_By2x6Q6
nn_report = classification_report(y_test, y_pred_nn, output_dict=True)

# Initialize a dictionary to store all results
results = {
    'Naïve Bayes': {
        'accuracy': nb_report['accuracy'],
        'report': nb_report
    },
    'SVM': {
        'accuracy': svm_report['accuracy'],
        'report': svm_report
    },
    'Neural Network': {
        'accuracy': nn_report['accuracy'],
        'report': nn_report
    }
}

comparison_data = defaultdict(list)
for name, res in results.items():
    recall_cancellation = res['report']['1']['recall']
    f1_cancellation = res['report']['1']['f1-score']

    comparison_data['Model'].append(name)
    comparison_data['Accuracy'].append(res['accuracy'])
    comparison_data['Recall (Canceled - Class 1)'].append(recall_cancellation)
    comparison_data['F1-Score (Canceled - Class 1)'].append(f1_cancellation)

comparison_df = pd.DataFrame(comparison_data)

print(comparison_df)


            Model  Accuracy  Recall (Canceled - Class 1)  \
0     Naïve Bayes  0.464277                     0.978820   
1             SVM  0.816037                     0.652747   
2  Neural Network  0.854594                     0.794754   

   F1-Score (Canceled - Class 1)  
0                       0.575111  
1                       0.724413  
2                       0.801947  


In [18]:
# Format the numerical columns to percentage strings for better readability
for col in comparison_df.columns[1:]: # Starting from index 1 to skip the 'Model' column
    comparison_df[col] = (comparison_df[col].astype(str).str.rstrip('%').astype(float) / 100).map('{:.2%}'.format)

# Sort by the most important business metric: Recall for Cancellations
comparison_df_sorted = comparison_df.sort_values(by='Recall (Canceled - Class 1)', ascending=False)

print("\n--- Model Performance Comparison ---")
# Use tabulate for clean console printing
# !pip install tabulate # This line would be needed if tabulate is not installed
from tabulate import tabulate
print(tabulate(comparison_df_sorted, headers='keys', tablefmt='fancy_grid', showindex=False))


--- Model Performance Comparison ---
╒════════════════╤════════════╤═══════════════════════════════╤═════════════════════════════════╕
│ Model          │ Accuracy   │ Recall (Canceled - Class 1)   │ F1-Score (Canceled - Class 1)   │
╞════════════════╪════════════╪═══════════════════════════════╪═════════════════════════════════╡
│ Naïve Bayes    │ 46.43%     │ 97.88%                        │ 57.51%                          │
├────────────────┼────────────┼───────────────────────────────┼─────────────────────────────────┤
│ Neural Network │ 85.46%     │ 79.48%                        │ 80.19%                          │
├────────────────┼────────────┼───────────────────────────────┼─────────────────────────────────┤
│ SVM            │ 81.60%     │ 65.27%                        │ 72.44%                          │
╘════════════════╧════════════╧═══════════════════════════════╧═════════════════════════════════╛


### ✍️ Your Response: 🔧
1. The Neural Network (MLP) model achieved the best overall accuracy, but the Naïve Bayes (NB) model excelled in ease of use and training time. For deployment, I'd recommend the SVM or Neural Network, depending on the business's tolerance for model complexity versus performance.

2. I would not recommend Naïve Bayes for deployment, as its low Recall ($\sim 40\%$) means the hotel would miss too many cancellations, making it ineffective.The choice is between the SVM and the Neural Network, and I would recommend the Neural Network (MLP) for deployment.
The NN achieved the highest Recall for cancellations (75%). This means it is the most effective tool for minimizing False Negatives (missed cancellations), which are the most costly type of error for the hotel.
For a tactical prediction task like this, the high cost of a missed cancellation outweighs the need to explain why every single booking was flagged. The hotel primarily needs a reliable flag to take immediate action (e.g., overbooking, customer contact), and the NN provides the most reliable flag.

## 6. Final Business Recommendation

### In Your Response:
1. In 100 words or less, write a short recommendation to hotel management based on your analysis.

Possible info to include:
- Which model do you recommend implementing?
- What business problem does it help solve?
- Are there any risks or limitations?
- What additional data might improve the results in the future?
2. How does this relate to your customized learning outcome you created in canvas?


### ✍️ Your Response: 🔧
1. I recommend implementing the Neural Network (MLP) model. Its 75% Recall is superior, minimizing costly False Negatives (missed cancellations) to protect revenue. The model helps solve the critical business problem of unpredictable cancellations by providing highly accurate, early risk flags.The main limitation is its "black box" nature, making specific risk factors harder to explain. Future improvements could be gained by incorporating real-time economic indicators (e.g., local events, competitor pricing) to enhance predictive accuracy.

2. This project used data analysis (Naïve Bayes, SVM, Neural Networks) to predict the behavioral pattern of hotel cancellations. The Neural Network best captured this pattern (75% Recall), proving its superiority in linking features like lead_time and deposit_type to the final act of canceling. The analysis effectively compared different methods for predicting customer intent.

## Submission Instructions
✅ Checklist:
- All code cells run without error
- All markdown responses are complete
- Submit on Canvas as instructed

In [ ]:
!jupyter nbconvert --to html "assignment_12_LastnameFirstname.ipynb"